Apple Stock (AAPL) Machine Learning Binary Classification

In [ ]:
#Import Libraries
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler

In [24]:
raw_df = yf.download('AAPL', start = '2010-01-01', end = '2025-07-26')
df = raw_df.copy() 

/var/folders/1p/bkw8vf0537qgpg1zgyfwq80r0000gn/T/ipykernel_93423/1667098571.py:1: FutureWarning: YF.download() has changed argument auto_adjust default to True
  raw_df = yf.download('AAPL', start = '2010-01-01', end = '2025-07-26')
[*********************100%***********************]  1 of 1 completed


In [25]:
raw_df.head()

Price,Close,High,Low,Open,Volume
Ticker,AAPL,AAPL,AAPL,AAPL,AAPL
Date,,,,,
2010-01-04,6.431896,6.446622,6.382907,6.414464,493729600
2010-01-05,6.443017,6.479383,6.409055,6.449629,601904800
2010-01-06,6.340531,6.468562,6.333919,6.443016,552160000
2010-01-07,6.328810,6.371487,6.282827,6.363974,477131200
2010-01-08,6.370886,6.371488,6.283129,6.320395,447610800


In [26]:
df.columns = df.columns.droplevel(1)

In [27]:
df.head()

Price,Close,High,Low,Open,Volume
Date,,,,,
2010-01-04,6.431896,6.446622,6.382907,6.414464,493729600
2010-01-05,6.443017,6.479383,6.409055,6.449629,601904800
2010-01-06,6.340531,6.468562,6.333919,6.443016,552160000
2010-01-07,6.328810,6.371487,6.282827,6.363974,477131200
2010-01-08,6.370886,6.371488,6.283129,6.320395,447610800


In [28]:
df = df.reset_index()
df = df[['Date', 'Close', 'Open', 'High', 'Low', 'Volume']]

In [29]:
df.head()

Price,Date,Close,Open,High,Low,Volume
0,2010-01-04,6.431896,6.414464,6.446622,6.382907,493729600
1,2010-01-05,6.443017,6.449629,6.479383,6.409055,601904800
2,2010-01-06,6.340531,6.443016,6.468562,6.333919,552160000
3,2010-01-07,6.328810,6.363974,6.371487,6.282827,477131200
4,2010-01-08,6.370886,6.320395,6.371488,6.283129,447610800


In [30]:
df['Target'] = (df['Close'].shift(-1) > df['Close']).astype(int)
df = df.dropna()
df.head()

Price,Date,Close,Open,High,Low,Volume,Target
0,2010-01-04,6.431896,6.414464,6.446622,6.382907,493729600,1
1,2010-01-05,6.443017,6.449629,6.479383,6.409055,601904800,0
2,2010-01-06,6.340531,6.443016,6.468562,6.333919,552160000,0
3,2010-01-07,6.328810,6.363974,6.371487,6.282827,477131200,1
4,2010-01-08,6.370886,6.320395,6.371488,6.283129,447610800,0


In [31]:
#1-day return to capture the momentum of the stock
df['Return_1d'] = df['Close'].pct_change()

In [32]:
#5-day return
df['Return_5d'] = df['Close'].pct_change(5)

In [33]:
#Moving averages for 10 days and 50 days, and their ratio
df['MA_10'] = df['Close'].rolling(10).mean()
df['MA_50'] = df['Close'].rolling(50).mean()
df['MA_ratio'] = df['MA_10'] / df['MA_50']

In [34]:
#5-day volatility (standard deviation of returns)
df['Volatility_5d'] = df['Return_1d'].rolling(5).std()

In [35]:
#Date-Based Features
df['DayOfWeek'] = df['Date'].dt.dayofweek
df['Month'] = df['Date'].dt.month